In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing libraies**

In [ ]:
import matplotlib.pyplot as plt
import scipy
import sys
import pickle
import librosa
import librosa.display as lpt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

# **Declare variables and functions**

In [ ]:
# Sample rate
sr = 45600

# Normalize function for Spectral rolloff
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

# Start and End for Zero Crossing-Rate
start = 1000
end = 1200

# **Load the paths of one song in each genre**

In [ ]:
blues_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/blues/blues.00000.wav'
classical_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/classical.00000.wav'
country_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/country/country.00000.wav'
disco_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/disco/disco.00000.wav'
hipHop_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/hiphop/hiphop.00000.wav'
jazz_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00000.wav'
metal_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/metal/metal.00000.wav'
pop_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/pop/pop.00000.wav'
raggae_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/reggae/reggae.00000.wav'
rock_sample = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/rock/rock.00000.wav'

# **Load one song from each genre**

In [ ]:
    # BLUES
data_blues, sr = librosa.load(blues_sample, sr=sr)
    # CLASSICAL
data_classical, sr = librosa.load(classical_sample, sr=sr)
    # COUNTRY
data_country, sr = librosa.load(country_sample, sr=sr)
    # DISCO
data_disco, sr = librosa.load(disco_sample, sr=sr)
    # HIP HOP
data_hipHop, sr = librosa.load(hipHop_sample, sr=sr)
    # JAZZ
data_jazz, sr = librosa.load(jazz_sample, sr=sr)
    # METAL
data_metal, sr = librosa.load(metal_sample, sr=sr)
    # POP
data_pop, sr = librosa.load(pop_sample, sr=sr)
    # RAGGAE
data_raggae, sr = librosa.load(raggae_sample, sr=sr)
    # ROCK
data_rock, sr = librosa.load(rock_sample, sr=sr)

songs = {
    'BLUES': data_blues,
    'CLASSICAL': data_classical,
    'COUNTRY': data_country,
    'DISCO': data_disco,
    'HIP HOP': data_hipHop,
    'JAZZ': data_jazz,
    'METAL': data_metal,
    'POP': data_pop,
    'RAGGAE': data_raggae,
    'ROCK': data_rock
}

# **Vizualizing audio files**

* **Waveforms**

In [ ]:
for genre, song in songs.items():
    plt.figure(figsize=(12, 4))
    lpt.waveplot(song)
    plt.title(genre)

* **Spectrograms**

1. Normal Spectrograms

In [ ]:
for genre, song in songs.items():
    stft = librosa.stft(song)
    plt.figure(figsize=(14, 6))
    lpt.specshow(stft, sr=sr, x_axis='time', y_axis='hz')
    plt.title(genre + ' Spectogram')
    plt.colorbar()

2. Mel Spectogram dB

In [ ]:
for genre, song in songs.items():
    stft = librosa.stft(song)
    magphase, _ = librosa.magphase(stft)
    mel_scale_sgram = librosa.feature.melspectrogram(S=magphase, sr=sr)
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)
    plt.figure(figsize=(14, 6))
    librosa.display.specshow(mel_sgram, sr=sr, x_axis='time', y_axis='mel')
    plt.title(genre + ' Mel Spectogram')
    plt.colorbar(format='%+2.0f dB')

3. Spectral Rollof

In [ ]:
for genre, song in songs.items():
    spectral_centroids = librosa.feature.spectral_centroid(song, sr=sr)[0]
    t = librosa.frames_to_time(range(len(spectral_centroids)))
    spectral_rollof = librosa.feature.spectral_rolloff(song+0.01, sr=sr)[0]
    plt.figure(figsize=(12, 4))
    lpt.waveplot(song, sr=sr, alpha=0.4)
    plt.plot(t, normalize(spectral_rollof), color='r')
    plt.title(genre + ' Spectral Rollof')

4. Chroma Feature

In [ ]:
for genre, song in songs.items():
    chroma = librosa.feature.chroma_stft(song, sr=sr)
    plt.figure(figsize=(16, 6))
    lpt.specshow(chroma, sr=sr, x_axis='time', y_axis='chroma', cmap='coolwarm')
    plt.colorbar()
    plt.title(genre + ' Chroma Feature')

5. Zero Crossing-Rate

In [ ]:
for genre, song in songs.items():
    plt.figure(figsize=(14, 5))
    plt.plot(song[start:end], color='#2B4F72')
    plt.title(genre + ' Zero Crossing-Rate')
    plt.grid()
    zero_crossings = librosa.zero_crossings(song[start:end], pad=False)
    print('Zero crossings ' + genre + ':', sum(zero_crossings))

6. Spectral Centroids

In [ ]:
for genre, song in songs.items():
    spectral_centroids = librosa.feature.spectral_centroid(song, sr=sr)[0]
    t = librosa.frames_to_time(range(len(spectral_centroids)))
    plt.figure(figsize=(12,4))
    lpt.waveplot(song, sr=sr, alpha=0.4)
    plt.plot(t, normalize(spectral_centroids), color='r')
    plt.title(genre + ' Spectral Centroids')

In [ ]:
for genre, song in songs.items():
    mfccs = librosa.feature.mfcc(song, sr=sr)
    print(mfccs.shape)
    plt.figure(figsize=(14, 6))
    librosa.display.specshow(mfccs, sr=sr, x_axis='time')
    plt.title(genre + ' MFCCS')